# a complex dashboard style interface for omnisci

1. define datasets
2. build indivudal plots
3. combine plots into a linked dashboard.

In [ ]:
import holoviews as hv
import pandas as pd
from holoviews import dim
import colorcet
import numpy as np
hv.extension('bokeh')

### As always, connect to the omnisci database with `ibis`

In [ ]:
import ibis

conn = ibis.omniscidb.connect(
    host='metis.mapd.com', user='demouser', password='HyperInteractive',
    port=443, database='mapd', protocol='https'
)

### We'll explore a complex application with the flights data table available in the omnisci repo.

In [ ]:
flights = conn.table("flights_donotmodify")

#### and select a sample of the data

In [ ]:
air = flights[flights.depdelay, flights.arrdelay, flights.carrier_name, flights.origin_state, flights.dep_timestamp]
air.head().execute()

### aggregate the data on the server
#### group by carrier name and compute the average of arrival delay and departure delay

In [ ]:
air_expr = air.groupby(
    air.carrier_name
).aggregate(
    arrdelay_avg=air.arrdelay.mean(), 
    depdelay_avg=air.depdelay.mean()
)
air_expr.head().execute()

### Connecting user events to omnisci databases.

### Using Holoviews Dataset to see what the data looks like.

we'll look at the arrivals and depatures of different airlines over time.

In [ ]:
from holoviews.operation.timeseries import resample

main_dataset = hv.Dataset(air, kdims=['dep_timestamp','arrdelay'])
agg_dataset = hv.Dataset(air_expr , kdims=['depdelay_avg', 'arrdelay_avg']).compute()

print(main_dataset)
print(agg_dataset)

### Visualizing the Holoviews Dataset

In [ ]:
summary_scatter = hv.Scatter(agg_dataset)
sel_stream = hv.streams.Selection1D(source=summary_scatter, index=[])

def tap_plot(index):
    carriers = list(agg_dataset.iloc[index]['carrier_name'])
    if not carriers:
        return hv.Curve(air.head(1).execute(), 'dep_timestamp', 'arrdelay')
    selection = main_dataset.select(carrier_name=carriers[0])
    timeseries = hv.Curve(selection, 'dep_timestamp', 'arrdelay').aggregate(function=np.mean).persist()
    return resample(timeseries, rule='H')

tap_dmap = hv.DynamicMap(tap_plot, streams=[sel_stream])

summary_scatter.opts(tools=['hover', 'tap'], size=10) + tap_dmap.opts(framewise=True)

### Another Holoviews Dataset example

In [ ]:
ds = hv.Dataset(flights)

origin_bars = (
    hv.Bars(ds, 'origin_city', 'flightnum')
    .aggregate('origin_city', np.count_nonzero)
    .sort('flightnum', reverse=True)
    .compute()
    .iloc[:10]
)
origin_bars.opts(xrotation=45, width=1000, height=300, show_grid=True)

#### What happens when there are errors in the the plot configuration??

`holoviews` gives useful error messages that make it possible to build visualization without referring to the docs.

In [ ]:
    try:
        origin_bars.opts(xrotation=45, width=1000, height=300, grid=True)
    except ValueError as E: print(E)

## Network graphs example

### achordgraph

from the `holoviews` docs.

> Chord is a special type of Graph which computes the locations of
    each node on a circle and the chords connecting them. The amount
    of radial angle devoted to each node and the number of chords are
    scaled by a weight supplied as a value dimension.

`connections` is a `holoviews` expressions in a form that a chord plot can be created with.

In [ ]:
connections = (
    ds.clone(kdims=['origin_city', 'dest_city', 'origin_merc_x', 'origin_merc_y', 'dest_merc_x', 'dest_merc_y'],
             vdims=['flightnum'])
    .select(origin_city=list(origin_bars.dimension_values('origin_city', expanded=False)))
    .aggregate(function=np.count_nonzero)
    .compute()
    .sort('flightnum', reverse=True)
    .iloc[:200]
)
connections.data.iloc[:2]

In [ ]:
chord = hv.Chord(connections.sort('origin_city'), ['origin_city', 'dest_city']).opts(
    edge_color='origin_city', node_color='index', cmap='Category20', tools=['hover', 'tap'],
    labels='index', width=500, height=500
)
chord

### plotting data onto maps

`holoviews` contains tools for different web map tile services.

In [ ]:
hv.element.tiles.Wikipedia() + hv.element.tiles.CartoLight() + hv.element.tiles.ESRI()

In [ ]:
segs = hv.Segments(connections, ['origin_merc_x', 'origin_merc_y', 'dest_merc_x', 'dest_merc_y'], 'flightnum')
locs = hv.Points(connections, ['origin_merc_x', 'origin_merc_y'], 'origin_city')

tiles = hv.element.tiles.Wikipedia()

(tiles
 * segs.opts(line_width=hv.dim('flightnum').norm()*10, width=500, height=500, xaxis=None, yaxis=None, alpha=0.5)
 * locs.opts(size=5, fill_color='white', color='black', tools=['hover'])
)

### putting the plots together

> Currently we're working off of two PRs so this is new development.  


In [ ]:
import panel as pn

ls = hv.link_selections.instance()

pn.Column(
    ls(origin_bars),
    pn.Row(chord, tiles * ls(segs) * ls(locs))
)

## another projection of the flights data

In [ ]:
import panel as pn

ls = hv.link_selections.instance()

ds = hv.Dataset(flights)

origin_points = (
    hv.Points(ds.clone(vdims=['flightnum'])
        .aggregate(['origin_city', 'origin_merc_x', 'origin_merc_y'], np.count_nonzero),
        ['origin_merc_x', 'origin_merc_y'], ['origin_city', 'flightnum']
    ).sort('flightnum', reverse=True)
    .compute()
    .iloc[:20]
).opts(xaxis=None, yaxis=None, width=800, height=800, size=5+hv.dim('flightnum').norm()*15, color='black', tools=['hover'])

tiles = hv.element.tiles.Wikipedia()

top_airports = list(origin_bars.dimension_values('origin_city', expanded=False))
top_airport_ds = ds.select(origin_city=top_airports)

hists = []
for delay in ('depdelay', 'securitydelay', 'lateaircraftdelay', 'weatherdelay'):
    delay_hist = top_airport_ds.hist(delay, adjoin=False, bin_range=(0, 60), normed=False).opts(frame_width=200, height=200)
    hists.append(ls(delay_hist))
    
pn.Column(
    '# Delay Explorer',
    pn.Row(
        hv.Layout(hists).cols(1),
        tiles * ls(origin_points)
    )
)

This is what it looks like when we put OmniSci, hvPlot, Holoviews, Pandas, GeoViews, Ibis, and Panel together. 

In [ ]:
    __import__("graphviz").Source("""graph {layout=circo 
    {{intake -- {ibis -- pandas}} -- {holoviews}}--hvplot geoviews--{holoviews hvplot}[style=dotted]}
    """)